<a href="https://colab.research.google.com/github/Rojerr9241/3DGRAVITY/blob/main/Si_IC_00_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estimación de la Temperatura de la Superficie Terrestre

## Preámbulo

In [ ]:
# --- 1. INSTALACIÓN E IMPORTACIONES NECESARIAS ---

# Instalar bibliotecas si es necesario (descomentar si es la primera vez)
# !pip install earthengine-api geemap geopandas -q

import ee
import geemap
import geopandas as gpd
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import date, timedelta
import datetime
import calendar # Necesario para calcular días en el mes
import matplotlib.pyplot as plt
import os # Necesario para la lógica de copia de archivos
from pathlib import Path  # Biblioteca moderna para manejar rutas

In [ ]:
# CHECKED
"""
Script para cargar y validar un GeoDataFrame desde Google Drive (para Colab)
o desde un directorio local.
"""
# --- 1. CONFIGURACIÓN ---
# Todas las variables que podrías necesitar cambiar están aquí.
DRIVE_MOUNT_POINT = '/content/drive'
SHARED_DRIVE_NAME = "Si_IC (Isla de Calor CAMe)"
FOLDER_NAME = "Datos"
FILE_NAME = "megalopolis_mun.geojson"
REQUIRED_COLUMNS = ['NOM_ENT', 'NOM_MUN']


# --- 2. FUNCIONES DE UTILIDAD (HELPERS) ---
# Funciones pequeñas y enfocadas que hacen una sola cosa.

def check_is_colab() -> bool:
    """Verifica si el script se ejecuta en Google Colab."""
    try:
        import google.colab
        return True
    except ImportError:
        return False

def _mount_google_drive() -> bool:
    """
    Monta Google Drive si está en Colab y es necesario.
    Es 'idempotente': se puede llamar varias veces, pero solo montará una vez.

    Returns:
        bool: True si Drive está listo (ya montado o montado con éxito).
              False si el montaje falló.
    """
    mount_path = Path(DRIVE_MOUNT_POINT)

    # 1. Verificar si ya está montado
    if mount_path.is_dir() and any(mount_path.iterdir()):
        print(f"Google Drive ya está montado en {DRIVE_MOUNT_POINT}.")
        return True

    # 2. Si no, intentar montar
    try:
        from google.colab import drive
        print(f"Montando Google Drive en {DRIVE_MOUNT_POINT}...")
        drive.mount(DRIVE_MOUNT_POINT)
        print("¡Drive montado exitosamente!")
        return True
    except Exception as e:
        print(f"ERROR: Fallo al montar Google Drive: {e}")
        return False

def _read_geodataframe(file_path: Path) -> gpd.GeoDataFrame | None:
    """
    Intenta leer un archivo GeoJSON desde la ruta.
    Maneja los errores de I/O (Input/Output), como archivo no encontrado o corrupto.
    """
    if not file_path.exists():
        print(f"\nERROR: El archivo no se encontró en la ruta: {file_path}")
        return None

    try:
        print(f"\nCargando GeoJSON desde: {file_path}")
        gdf = gpd.read_file(file_path)
        print("¡GeoDataFrame cargado exitosamente!")
        return gdf
    except Exception as e:
        print(f"\nERROR al leer el archivo GeoJSON: {e}")
        return None

def _validate_gdf_columns(gdf: gpd.GeoDataFrame) -> bool:
    """
    Valida que el GeoDataFrame contenga las columnas requeridas.
    """
    current_columns = set(gdf.columns)
    required = set(REQUIRED_COLUMNS)
    missing_cols = required - current_columns

    if missing_cols:
        print(f"ERROR: El GeoJSON no contiene las columnas requeridas: {missing_cols}")
        return False

    print("Validación de columnas exitosa.")
    return True


# --- 3. FUNCIONES PRINCIPALES (ORQUESTADORES) ---
# Funciones de alto nivel que coordinan a los helpers.

def get_file_path(is_colab_env: bool) -> Path | None:
    """
    Orquesta la obtención de la ruta del archivo, manejando el montaje de Drive.

    Returns:
        Un objeto Path al archivo, o None si la configuración falla.
    """
    if is_colab_env:
        print("Entorno de Google Colab detectado.")

        # 1. Delegar la tarea compleja
        drive_is_ready = _mount_google_drive()

        # 2. Comprobar el resultado
        if not drive_is_ready:
            print("No se puede continuar sin montar Drive.")
            return None

        # 3. Construir la ruta (la tarea principal de esta función)
        return (
            Path(DRIVE_MOUNT_POINT)
            / "Shared drives"
            / SHARED_DRIVE_NAME
            / FOLDER_NAME
            / FILE_NAME
        )

    else:
        # Lógica para entorno local
        print("WARNING: No se detectó Google Colab. Asumiendo archivo local.")
        return Path() / FILE_NAME

def load_and_validate_gdf(file_path: Path) -> gpd.GeoDataFrame | None:
    """
    Orquesta la carga y validación del GeoDataFrame.
    """
    # 1. Delegar la lectura del archivo
    gdf = _read_geodataframe(file_path)

    if gdf is None:
        return None # Falló la lectura, no continuar.

    # 2. Delegar la validación de las columnas
    is_valid = _validate_gdf_columns(gdf)

    if is_valid:
        return gdf
    else:
        return None # Falló la validación

def setup_geodata() -> gpd.GeoDataFrame | None:
    """
    Función principal para orquestar todo el proceso de carga.
    1. Revisa el entorno.
    2. Obtiene la ruta del archivo.
    3. Carga y valida el archivo.
    """
    print("--- Iniciando configuración de datos ---")
    #is_colab = check_is_colab()
    file_path = get_file_path(True)

    if file_path:
        gdf = load_and_validate_gdf(file_path)
        return gdf
    else:
        print("\nNo se pudo determinar la ruta del archivo. No se puede cargar el GeoDataFrame.")
        return None


# --- 4. EJECUCIÓN ---
# Esta es la única parte del script que "corre" al inicio.
# Todas las demás son definiciones de funciones.

gdf = setup_geodata()

if gdf is not None:
    print("\n--- ¡Configuración completada! GeoDataFrame está listo. ---")
    print("Primeras 5 filas del GeoDataFrame:")
    print(gdf.head())
else:
    print("\n--- La carga de datos falló. Revisa los mensajes de error. ---")

--- Iniciando configuración de datos ---
Entorno de Google Colab detectado.
Google Drive ya está montado en /content/drive.

Cargando GeoJSON desde: /content/drive/Shared drives/Si_IC (Isla de Calor CAMe)/Datos/megalopolis_mun.geojson
¡GeoDataFrame cargado exitosamente!
Validación de columnas exitosa.

--- ¡Configuración completada! GeoDataFrame está listo. ---
Primeras 5 filas del GeoDataFrame:
  CVE_ENT CVE_MUN CVEGEO                 NOMGEO           NOM_ENT  \
0      09     002  09002           Azcapotzalco  Ciudad de México   
1      09     003  09003               Coyoacán  Ciudad de México   
2      09     004  09004  Cuajimalpa de Morelos  Ciudad de México   
3      09     005  09005      Gustavo A. Madero  Ciudad de México   
4      09     006  09006              Iztacalco  Ciudad de México   

                 NOM_MUN                                           geometry  
0           Azcapotzalco  MULTIPOLYGON (((2792538.869 838006.936, 279258...  
1               Coyoacán  MULT

In [ ]:
# Nombre de la carpeta en Google Drive donde GEE guarda los resultados (DEBE EXISTIR)
DRIVE_FOLDER = 'GEE_TSM_Exports'
# --- 4. CONFIGURACIÓN DE WIDGETS INTERACTIVOS ---

# Crear un objeto de mapa interactivo global que se usará para la visualización
Map = geemap.Map(center=[20.5888, -100.3899], zoom=7)

# 4.1 Controles de Ubicación
if gdf is not None:
    entidades_unicas = sorted(gdf['NOM_ENT'].unique())
else:
    entidades_unicas = ['ERROR: GeoJSON no cargado']

entidad_dropdown = widgets.Dropdown(
    options=entidades_unicas,
    description='1. Entidad:',
    disabled=(gdf is None),
    style={'description_width': 'initial'}
)

municipio_dropdown = widgets.Dropdown(
    options=[],
    description='2. Municipio:',
    disabled=(gdf is None),
    style={'description_width': 'initial'}
)

# Función de "Observador" que mantiene la lógica de cascada (Entidad -> Municipio)
def on_entidad_change(change):
    if change['type'] == 'change' and change['name'] == 'value' and gdf is not None:
        entidad_seleccionada = change['new']
        municipios_df = gdf[gdf['NOM_ENT'] == entidad_seleccionada]
        municipios_unicos = sorted(municipios_df['NOM_MUN'].unique())
        municipio_dropdown.options = municipios_unicos
        municipio_dropdown.disabled = False
        with output:
            clear_output()
            print(f"Municipios disponibles para {entidad_seleccionada} cargados.")

# Conectar el observador de Entidad
entidad_dropdown.observe(on_entidad_change)


# 4.2 Controles de Temporalidad, Periodicidad y Exportación
fecha_inicial_picker = widgets.DatePicker(
    value=date(2023, 1, 1), # Valor inicial recomendado
    description='3. Fecha Inicial:',
    disabled=False,
    style={'description_width': 'initial'}
)

fecha_final_picker = widgets.DatePicker(
    value=date(2023, 12, 31), # Valor inicial recomendado
    description='4. Fecha Final:',
    disabled=False,
    style={'description_width': 'initial'}
)

periodicidad_radio = widgets.RadioButtons(
    options=['Semanal', 'Mensual', 'Anual'],
    description='5. Periodicidad:',
    value='Mensual', # Valor por defecto
    disabled=False,
    style={'description_width': 'initial'}
)

# Campo de texto para la ruta de copia adicional
# Se recomienda usar rutas relativas a MyDrive, e.g., 'Carpeta_Adicional/Resultados_TSM'
# copy_path_textbox = widgets.Text(
#     value='',
#     description='6. Ruta de Copia (Opcional):',
#     placeholder='Ej: MiDrive/Mi_Proyecto/Resultados',
#     disabled=False,
#     layout=widgets.Layout(width='auto'),
#     style={'description_width': 'initial'}
# )

copy_path_textbox = widgets.Text(
    value=DRIVE_FOLDER,  # Usar tu constante como valor por defecto
    description='6. Carpeta en Drive:', # <-- Descripción mucho más clara
    placeholder='Ej: GEE_Resultados/Mi_Proyecto',
    disabled=False,
    layout=widgets.Layout(width='auto'),
    style={'description_width': 'initial'}
)


# 4.3 Botón de Ejecución
ejecutar_button = widgets.Button(
    description="Ejecutar Visualización GEE",
    button_style='success',
    icon='earth'
)

# Contenedor de Salida
output = widgets.Output()

In [ ]:


GEE_PROJECT_ID = 'uhi-project-data-processing'

# --- 2.1 PARÁMETROS DE EXPORTACIÓN Y COPIA ---
# Cambia esta variable a True si deseas exportar los rásters a Google Drive
EXPORTAR_A_DRIVE = False
# Nombre de la carpeta en Google Drive donde GEE guarda los resultados (DEBE EXISTIR)
DRIVE_FOLDER = 'GEE_TSM_Exports'

# --- 3. FUNCIONES CORE DE EARTH ENGINE ---

# Parámetros de Visualización para la TSM
VIS_PARAMS_TSM = {
    'min': 15,
    'max': 40,
    'palette': [
        '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
        '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
        '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
        'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
        'ff0000', 'de0101', 'c21301', 'a71001', '911001'
    ]
}

# --- 2. FUNCIONES DE UTILIDAD (HELPERS) ---

def initialize_gee(project_id: str) -> bool:
    """
    Autentica (si es necesario) e inicializa Google Earth Engine.

    Returns:
        bool: True si GEE se inicializó con éxito, False si falló.
    """
    try:
        # 1. Intento optimista:
        # Intenta inicializar usando credenciales existentes (cacheadas).
        print("Intentando inicializar Earth Engine...")
        ee.Initialize(project=project_id) # 1. Fails
        print("¡Earth Engine inicializado exitosamente (usando credenciales existentes)!")
        return True

    except Exception as e:
        # 2. Si falla, es probable que no haya credenciales.
        print(f"Inicialización fallida ({e}). Se requiere autenticación.")

        try:
            # 3. Forzar la autenticación (esto abrirá una ventana/prompt)
            print("Por favor, sigue las instrucciones de autenticación...")
            ee.Authenticate() # 3. User clicks "Cancel"

            # 4. Reintentar la inicialización después de autenticar
            ee.Initialize(project=project_id) # 4. Fails *again*
            print("¡Earth Engine autenticado e inicializado exitosamente!")
            return True

        except Exception as auth_e: # We catch the 2nd failure!
            # 5. Si la autenticación o el segundo intento fallan
            print(f"ERROR: No se pudo autenticar o inicializar GEE: {auth_e}")
            return False

# FUNCTIONS
def enmascarar_nubes(image):
    """
    Función para enmascarar nubes y sombras de nubes.
    """
    qa = image.select('QA_PIXEL')
    cloud_shadow_bit_mask = (1 << 3)
    cloud_bit_mask = (1 << 5)

    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0).And(
           qa.bitwiseAnd(cloud_bit_mask).eq(0))

    return image.updateMask(mask)

# --- Pega esto en tu Celda [1] y reemplaza la antigua 'enmascarar_nubes' ---

# def enmascarar_nubes(image):
#     """
#     Aplica factores de escala y enmascara nubes/sombras usando la
#     banda QA_PIXEL de Landsat 8/9 C2 L2.
#     """

#     # 1. Aplicar factores de escala (¡Muy importante!)
#     # Bandas ópticas (SR)
#     opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
#     # Banda térmica (ST)
#     thermalBand = image.select('ST_B10').multiply(0.00341802).add(149.0)

#     # 2. Enmascarar nubes y sombras usando la banda QA_PIXEL
#     qa = image.select('QA_PIXEL')

#     # Bits que queremos enmascarar (marcar como 'malos')
#     dilatedCloud = 1 << 1  # 2
#     cloud = 1 << 3         # 8
#     cloudShadow = 1 << 4   # 16

#     # Crear una máscara binaria.
#     # Los píxeles 'malos' (nube, sombra) se marcan con 1.
#     badPixels = qa.bitwiseAnd(dilatedCloud).neq(0) \
#                   .Or(qa.bitwiseAnd(cloud).neq(0)) \
#                   .Or(qa.bitwiseAnd(cloudShadow).neq(0))

#     # Invertir la máscara.
#     # Queremos mantener los píxeles que NO son 'malos' (mask = 0).
#     mask = badPixels.Not()

#     # 3. Devolver la imagen escalada Y enmascarada
#     return image.addBands(opticalBands, None, True) \
#                 .addBands(thermalBand, None, True) \
#                 .updateMask(mask)

def calcular_tsm(image, roi):
    """
    Función para calcular la Temperatura de la Superficie Terrestre (TSM) en Celsius.
    Requiere el 'roi' para calcular el min/max de NDVI en la región.
    """
    # 1. Seleccionar Bandas Ópticas y Térmicas y aplicar factores de escala
    optical_bands = image.select('SR_B4', 'SR_B5').multiply(0.0000275).add(-0.2)
    thermal_band = image.select('ST_B10').multiply(0.00341802).add(149.0)

    image_procesada = image.addBands(optical_bands.rename(['Red', 'NIR'])) \
                          .addBands(thermal_band.rename(['Brightness_Temp_K']))

    # 2. Calcular NDVI
    ndvi = image_procesada.normalizedDifference(['NIR', 'Red']).rename('NDVI')
    image_procesada = image_procesada.addBands(ndvi)

    # 3. Calcular Emisividad
    # Reducir para encontrar min/max de NDVI en la RDI
    min_max_ndvi = ndvi.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=roi,
        scale=30,
        maxPixels=1e9
    )

    min_val = ee.Number(min_max_ndvi.get('NDVI_min'))
    max_val = ee.Number(min_max_ndvi.get('NDVI_max'))

    # --- CORRECCIÓN DE ERROR NULO (Añadido) ---
    # Si min_val es nulo (es decir, la imagen está completamente enmascarada/nublada en la ROI),
    # devolvemos la imagen sin la banda LST_C para que la mediana ignore esta imagen.
    es_valido = min_val.gt(-99999).And(max_val.gt(-99999)) # Usar una comprobación simple de no-nulo

    def calcular_y_añadir_tsm(min_val, max_val):
        # Fracción de Vegetación (PV)
        pv = ndvi.subtract(min_val).divide(max_val.subtract(min_val)).pow(2).rename('PV')

        # Emisividad
        emisividad = pv.multiply(0.004).add(0.986).rename('EMISSIVITY')

        # 4. Calcular la Temperatura de la Superficie Terrestre en Celsius
        brightness_temp_k = image_procesada.select('Brightness_Temp_K')
        lambda_term = ee.Image(0.00115).multiply(brightness_temp_k).divide(ee.Number(1.4388))

        tsm = brightness_temp_k.divide(
            ee.Image(1).add(
                lambda_term.multiply(emisividad.log())
            )
        ).subtract(273.15).rename('LST_C') # Kelvin a Celsius (-273.15)

        return image.addBands(tsm)

    # ee.Algorithms.If verifica si es_valido es True
    return ee.Algorithms.If(
        es_valido,
        calcular_y_añadir_tsm(min_val, max_val),
        image.select([]) # Si no es válido, devuelve una imagen vacía (o sin la banda TSM)
    )

# EXPORT TO DRIVE
def exportar_imagen_a_drive(image, roi, filename, folder):
    """Configura y comienza la tarea de exportación a Google Drive."""
    task = ee.batch.Export.image.toDrive(
        image=image.select('LST_C').float(), # Exportar solo la banda TSM como float
        description=filename,
        folder=folder,
        fileNamePrefix=filename,
        region=roi.bounds().getInfo()['coordinates'], # Usar la caja delimitadora de la ROI
        scale=30, # Resolución de Landsat
        crs='EPSG:4326', # Sistema de referencia de coordenadas
        maxPixels=1e10
    )
    task.start()
    return f"  -> EXPORTACIÓN INICIADA: '{filename}'. Verifique la pestaña Tareas (Tasks) de GEE."
# --- Celda [1] (Reemplaza tu función de exportación) ---

# def exportar_imagen_a_drive(imagen, roi, nombre_capa, id_carpeta_drive):
#     """
#     Función de exportación unificada.
#     Inicia una tarea de GEE al ID de la carpeta de Drive especificada.

#     Args:
#         imagen (ee.Image): La imagen a exportar.
#         roi (ee.Geometry): La región de interés.
#         nombre_capa (str): El nombre para el archivo y la tarea.
#         id_carpeta_drive (str): El ID de la carpeta de destino
#                                 (de "Mi unidad" o "Unidad compartida").
#     """
#     try:
#         nombre_limpio = nombre_capa.replace(' ', '_').replace('(', '').replace(')', '').replace(':', '_').replace('-', '_')

#         print(f"   Iniciando exportación para: {nombre_limpio}")

#         task = ee.batch.Export.image.toDrive(
#             image=imagen.toFloat(),
#             description=nombre_limpio,
#             driveFolderId=id_carpeta_drive, # <-- ¡ESTE ES EL CAMBIO CLAVE!
#             # folder=... (Este ya no se usa)
#             fileNamePrefix=nombre_limpio,
#             region=roi,
#             scale=30,
#             maxPixels=1e10
#         )

#         task.start()
#         print(f"   > ¡Tarea '{nombre_limpio}' iniciada! Revise la pestaña 'Tasks' de GEE.")

#     except Exception as e:
#         print(f"   > ERROR al iniciar la exportación para '{nombre_limpio}': {e}")

def procesar_capa(coleccion_filtrada, roi, nombre_capa, visibilidad=False, target_copy_folder=""): # ** Check
    """Aplica el procesamiento de nubes y TSM, calcula la mediana y añade la capa al mapa."""

    # --- DEBUGGING ---
    print('1. Imágenes iniciales (coleccion_filtrada):', coleccion_filtrada.size().getInfo())

    # Aplica la máscara de nubes
    coleccion_enmascarada = coleccion_filtrada.map(enmascarar_nubes)
    print('2. Imágenes después de enmascarar (mismo tamaño):', coleccion_enmascarada.size().getInfo())

    # Calcula TSM
    coleccion_con_tsm = coleccion_enmascarada.map(lambda img: calcular_tsm(img, roi))
    print('3. Imágenes después de calcular_tsm (mismo tamaño):', coleccion_con_tsm.size().getInfo())

    # ¡EL FILTRO! Aquí es donde probablemente desaparecen
    #tsm_coleccion = coleccion_con_tsm.filter(ee.Filter.notNull(['LST_C']))
    tsm_coleccion = coleccion_con_tsm.filter(ee.Filter.listContains('system:band_names', 'LST_C'))
    print('4. Imágenes FINALES (después de filtrar):', tsm_coleccion.size().getInfo())
    # --- FIN DEBUGGING ---

    # La función lambda debe capturar 'roi' para pasarlo a 'calcular_tsm'
    # tsm_coleccion = coleccion_filtrada.map(enmascarar_nubes).map(
    #     lambda img: calcular_tsm(img, roi)
    # ).filter(ee.Filter.notNull(['LST_C'])) # Eliminar imágenes que fallaron el cálculo LST (devolvieron vacío)

    # Si la colección resultante está vacía después de filtrar, median() fallará.
    # Comprobamos el tamaño de la colección después del filtro.
    if tsm_coleccion.size().getInfo() == 0:
        return f"  -> Capa '{nombre_capa}' OMITIDA: No hay imágenes válidas sin nubes para este periodo."

    mediana_tsm = tsm_coleccion.select('LST_C').median()

    # Añadir la capa al mapa global de geemap
    Map.addLayer(mediana_tsm.clip(roi), VIS_PARAMS_TSM, nombre_capa, visibilidad)

    filename_prefix = nombre_capa.replace(' ', '_').replace('(', '').replace(')', '').replace('-', '_')
    log_msg = f"  -> Capa '{nombre_capa}' añadida (visible: {visibilidad})."

    # # Si la exportación está habilitada, iniciar la tarea
    # if EXPORTAR_A_DRIVE:
    #     export_msg = exportar_imagen_a_drive(mediana_tsm.clip(roi), roi, filename_prefix, DRIVE_FOLDER)
    #     log_msg += f"\n{export_msg}"

    #     # Iniciar el proceso de copia
    #     copy_msg = copiar_archivo_a_ruta_secundaria(filename_prefix, target_copy_folder)
    #     log_msg += f"\n{copy_msg}"

    return log_msg

# --- 5. FUNCIONES AUXILIARES DE PROCESAMIENTO ---
# (Coloca estas funciones ANTES de la función principal)

def _obtener_roi_desde_gdf(entidad, municipio, gdf): # DONE
    """
    Filtra el GeoDataFrame y convierte la geometría del municipio en un ee.Geometry (ROI).
    Si falla, lanza una Excepción.
    """
    print("--- 2. Creando ROI (ee.Geometry) ---")
    municipio_gdp = gdf[
        (gdf['NOM_ENT'] == entidad) &
        (gdf['NOM_MUN'] == municipio)
    ]

    if municipio_gdp.empty:
        raise Exception(f"No se pudo encontrar la geometría para: {municipio}, {entidad}")

    # Convertimos la geometría de GeoPandas a ee.Geometry
    municipio_gdp_wgs84 = municipio_gdp.to_crs(epsg=4326)
    geom_shapely = municipio_gdp_wgs84.iloc[0].geometry

    try:
        if geom_shapely.geom_type == 'MultiPolygon':
            roi = ee.Geometry.MultiPolygon(geom_shapely.__geo_interface__['coordinates'])
        elif geom_shapely.geom_type == 'Polygon':
            roi = ee.Geometry.Polygon(geom_shapely.__geo_interface__['coordinates'])
        else:
            print("ADVERTENCIA: Geometría no es Polígono ni MultiPolígono. Usando geemap.geopandas_to_ee.")
            roi = geemap.geopandas_to_ee(municipio_gdp_wgs84)

        print("ROI cargada en Earth Engine.")
        return roi

    except Exception as e:
        print(f"ERROR CRÍTICO al convertir Shapely a ee.Geometry: {e}")
        print("Usando el método genérico de geemap como alternativa.")
        try:
            roi = geemap.geopandas_to_ee(municipio_gdp_wgs84)
            print("ROI cargada (método alternativo).")
            return roi
        except Exception as e_alt:
            raise Exception(f"Fallo del método alternativo: {e_alt}")


def _obtener_coleccion_landsat(roi, fecha_ini, fecha_fin): # DONE
    """
    Filtra la colección de Landsat 8 por fechas, límites (ROI) y nubes.
    Si no encuentra imágenes, lanza una Excepción.
    """
    print("\n--- 3. Filtrando Colección Landsat 8 ---")
    fecha_inicio_str = fecha_ini.strftime('%Y-%m-%d')
    fecha_fin_str = fecha_fin.strftime('%Y-%m-%d')

    coleccion_completa = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterBounds(roi) \
        .filterDate(fecha_inicio_str, fecha_fin_str) \
        .filterMetadata('CLOUD_COVER_LAND', 'less_than', 30)

    count = coleccion_completa.size().getInfo()
    print(f"Imágenes Landsat 8 encontradas en el rango: {count}")

    if count == 0:
        raise Exception("No se encontraron imágenes en el rango y RDI especificados.")

    return coleccion_completa, fecha_inicio_str, fecha_fin_str


def _procesar_por_periodicidad(coleccion, roi, periodo, start_dt, end_dt, municipio_sel, target_copy_folder):
    """
    Toma la colección completa y la procesa según la periodicidad (Semanal, Mensual, Anual).
    Llama a 'procesar_capa' para cada sub-periodo.
    """
    print(f"\n--- 4. Procesamiento de TSM por Periodo: {periodo} ---")
    resultados_procesamiento = []

    # NOTA: Asumimos que la función 'procesar_capa' existe en tu código.
    # procesar_capa(coleccion_periodo, roi, nombre_capa, exportar_flag, target_copy_folder)

    if periodo == 'Mensual':
        # --- PROCESAMIENTO MENSUAL ---
        nombres_meses = ['', 'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
        current_dt = start_dt.replace(day=1)

        while current_dt <= end_dt:
            current_year = current_dt.year
            current_month = current_dt.month
            days_in_month = calendar.monthrange(current_year, current_month)[1]
            end_filter_dt = current_dt + timedelta(days=days_in_month)

            start_date_str_p = current_dt.strftime('%Y-%m-%d')

            if end_filter_dt > end_dt:
                end_date_str_exclusive = (end_dt + timedelta(days=1)).strftime('%Y-%m-%d')
                nombre_capa = f'TSM_Mediana_{nombres_meses[current_month]}_{current_year}_Parcial_{municipio_sel}'
            else:
                end_date_str_exclusive = end_filter_dt.strftime('%Y-%m-%d')
                nombre_capa = f'TSM_Mediana_{nombres_meses[current_month]}_{current_year}_{municipio_sel}'

            coleccion_periodo = coleccion.filterDate(start_date_str_p, end_date_str_exclusive)

            if coleccion_periodo.size().getInfo() > 0:
                msg = procesar_capa(coleccion_periodo, roi, nombre_capa, False, target_copy_folder)
                resultados_procesamiento.append(msg)

            current_dt = end_filter_dt

    elif periodo == 'Anual':
        # --- PROCESAMIENTO ANUAL ---
        nombre_capa = f'TSM_Mediana_Anual_{start_dt.strftime("%Y-%m-%d")}_a_{end_dt.strftime("%Y-%m-%d")}_{municipio_sel}'
        if coleccion.size().getInfo() > 0:
            msg = procesar_capa(coleccion, roi, nombre_capa, True, target_copy_folder)
            resultados_procesamiento.append(msg)

    elif periodo == 'Semanal':
        # --- PROCESAMIENTO SEMANAL ---
        current_date = start_dt
        week_index = 1
        while current_date <= end_dt:
            start_week = current_date
            end_week_inclusive = current_date + timedelta(days=6)
            if end_week_inclusive > end_dt:
                end_week_inclusive = end_dt

            start_str_p = start_week.strftime('%Y-%m-%d')
            end_str_exclusive = (end_week_inclusive + timedelta(days=1)).strftime('%Y-%m-%d')
            end_str_inclusive = end_week_inclusive.strftime('%Y-%m-%d')

            coleccion_periodo = coleccion.filterDate(start_str_p, end_str_exclusive)

            if coleccion_periodo.size().getInfo() > 0:
                nombre_capa = f'TSM_Semana_{week_index}_{start_str_p}_a_{end_str_inclusive}_{municipio_sel}'
                msg = procesar_capa(coleccion_periodo, roi, nombre_capa, False, target_copy_folder)
                resultados_procesamiento.append(msg)

            current_date = current_date + timedelta(days=7)
            week_index += 1
    else:
        raise Exception(f"Periodicidad '{periodo}' no reconocida.")

    return resultados_procesamiento


def _agregar_capa_color_verdadero(coleccion, roi, fecha_ini_str, fecha_fin_str, municipio_sel, ruta_copia, exportar, carpeta_drive):
    """
    Genera y añade la capa de referencia de Color Verdadero al mapa.
    También gestiona su exportación y copia secundaria.

    NOTA: Asumimos que existen 'enmascarar_nubes', 'exportar_imagen_a_drive' y 'copiar_archivo_a_ruta_secundaria'.
    """
    print("\n--- 5. Generando Capa de Color Verdadero ---")
    composicion_anual_sr = coleccion.map(enmascarar_nubes).median()

    vis_params_color_verdadero = {
        'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.0, 'max': 0.2, 'gamma': 1.4,
    }

    nombre_ref = f'Color_Verdadero_Mediana_{fecha_ini_str}_a_{fecha_fin_str}_{municipio_sel}'
    Map.addLayer(composicion_anual_sr.clip(roi), vis_params_color_verdadero, nombre_ref, True)

    if exportar:
      pass
        # exportar_imagen_a_drive(composicion_anual_sr.clip(roi), roi, nombre_ref, carpeta_drive)
        # print(f"Capa de Color Verdadero añadida y exportación iniciada.")

        # # # Limpiar nombre para la copia (esto debería estar en una función de utilidad)
        # # nombre_archivo_limpio = nombre_ref.replace(' ', '_').replace('(', '').replace(')', '').replace('-', '_')
        # # copy_msg = copiar_archivo_a_ruta_secundaria(nombre_archivo_limpio, ruta_copia)
        # # print(copy_msg)
    else:
        print(f"Capa de Color Verdadero añadida (visible).")

# MAIN

# --- 6. FUNCIÓN PRINCIPAL DE EJECUCIÓN (LÓGICA UNIFICADA) ---
# Esta es la nueva función "directora de orquesta"

def ejecutar_analisis_gee(b):
    """
    Función que se ejecuta al presionar el botón:
    1. Limpia la salida y valida los widgets.
    2. Llama a las funciones auxiliares para:
        a. Obtener la ROI.
        b. Obtener la colección de imágenes.
        c. Procesar por periodicidad (TSM).
        d. Añadir capa de color verdadero.
    3. Muestra el mapa final.
    """
    # 1. Limpiar salida y Recolectar valores
    with output:
        clear_output()
        print("--- 1. Iniciando Proceso ---")

        entidad_sel = entidad_dropdown.value
        municipio_sel = municipio_dropdown.value
        fecha_ini_sel = fecha_inicial_picker.value
        fecha_fin_sel = fecha_final_picker.value
        periodo_sel = periodicidad_radio.value
        ruta_copia_secundaria = copy_path_textbox.value.strip()

        # Validar entradas
        if not all([entidad_sel, municipio_sel, fecha_ini_sel, fecha_fin_sel]):
            print("ERROR: Por favor, asegúrate de seleccionar Entidad, Municipio, Fecha Inicial y Fecha Final.")
            return

        if fecha_fin_sel < fecha_ini_sel:
             print("ERROR: La Fecha Final no puede ser anterior a la Fecha Inicial.")
             return

        print(f"Ubicación: {municipio_sel}, {entidad_sel}")
        print(f"Rango: {fecha_ini_sel} a {fecha_fin_sel}")
        print(f"Periodicidad: {periodo_sel}")
        print(f"Exportación GEE (Carpeta principal): {'Sí' if EXPORTAR_A_DRIVE else 'No'} -> '{DRIVE_FOLDER}'")

        # Advertencias de copia (lógica de UI)
        if EXPORTAR_A_DRIVE and not ruta_copia_secundaria:
            print("NOTA: La exportación principal está activa, pero la ruta de copia secundaria está vacía.")
        elif ruta_copia_secundaria and not EXPORTAR_A_DRIVE:
            print("ADVERTENCIA: Se especificó una ruta de copia, pero 'EXPORTAR_A_DRIVE' es False. Se omitirá.")


        # --- INICIO DE LÓGICA DE PROCESAMIENTO ---
        try:
            # 2. Obtener ROI (Paso de Geometría)
            roi = _obtener_roi_desde_gdf(entidad_sel, municipio_sel, gdf)

            # Centrar y mostrar ROI en el mapa (esto es parte de la UI)
            Map.centerObject(roi, 9)
            Map.addLayer(roi, {'color': 'red'}, f'RDI: {municipio_sel}', True)

            # 3. Obtener Colección (Paso de Datos GEE)
            coleccion, fecha_ini_str, fecha_fin_str = _obtener_coleccion_landsat(roi, fecha_ini_sel, fecha_fin_sel)

            # # 4. Procesar TSM (Paso de Lógica de Negocio)
            # resultados = _procesar_por_periodicidad(
            #     coleccion, roi, periodo_sel,
            #     fecha_ini_sel, fecha_fin_sel,
            #     municipio_sel, ruta_copia_secundaria
            # )

            # # Imprimir resultados del procesamiento
            # if not resultados:
            #     print("ADVERTENCIA: No se generaron capas de TSM para el periodo y filtro seleccionados.")
            # else:
            #     for res in resultados:
            #         print(res)
            #     if EXPORTAR_A_DRIVE:
            #         print("\n**ATENCIÓN EXPORTACIÓN**:")
            #         print("1. Revise la pestaña 'Tasks' (Tareas) de GEE para ejecutar las exportaciones.")
            #         print("2. La COPIA SECUNDARIA requiere que ejecute los comandos `!cp ...` *después* de que GEE termine.")

            # # 5. Añadir Capa de Referencia (Paso de Visualización)
            # _agregar_capa_color_verdadero(
            #     coleccion, roi,
            #     fecha_ini_str, fecha_fin_str,
            #     municipio_sel, ruta_copia_secundaria,
            #     EXPORTAR_A_DRIVE, DRIVE_FOLDER
            # )

            # # 6. Mostrar Mapa en Colab
            # print("\n--- 6. Mostrando Mapa Interactivo ---")
            # display(Map)
            # print("El mapa interactivo se ha generado. Usa el panel de 'Capas' para alternar la visibilidad.")

        except Exception as e:
            # Capturamos cualquier error de las funciones auxiliares
            print(f"\n--- ERROR DURANTE LA EJECUCIÓN ---")
            print(f"Error: {e}")
            print("El proceso se ha detenido.")
            return

# Conectar el botón a la nueva función de análisis
ejecutar_button.on_click(ejecutar_analisis_gee)


gee_ready = initialize_gee(GEE_PROJECT_ID)

if gee_ready:
    # YES! It's 100% safe to run GEE code here.
    #image = ee.Image('LANDSAT/...')
    print("GEE inicializado y listo para usar.")

else:
    # NO! Don't run GEE code.
    print("Skipping GEE analysis.")

Intentando inicializar Earth Engine...
¡Earth Engine inicializado exitosamente (usando credenciales existentes)!
GEE inicializado y listo para usar.


In [ ]:

# --- 6. DESPLIEGUE DE LA INTERFAZ ---

print("\n--- Interfaz de Visualización y Parámetros ---")
print("Selecciona las opciones y presiona 'Ejecutar Visualización GEE'.")
if gdf is None:
    print("Por favor, soluciona el error de carga del GeoJSON en la Sección 2 para continuar.")
else:
    # Agrupar los controles en un layout vertical
    controles = widgets.VBox([
        entidad_dropdown,
        municipio_dropdown,
        widgets.HBox([fecha_inicial_picker, fecha_final_picker]), # Fechas en una línea
        periodicidad_radio,
        copy_path_textbox, # NUEVO: Campo de texto para la ruta de copia
        ejecutar_button
    ])

    display(controles, output)

    # Inicializar el dropdown de municipio con la primera entidad para que no esté vacío
    if entidades_unicas and entidades_unicas[0] != 'ERROR: GeoJSON no cargado':
        on_entidad_change({'type': 'change', 'name': 'value', 'new': entidades_unicas[0]})


--- Interfaz de Visualización y Parámetros ---
Selecciona las opciones y presiona 'Ejecutar Visualización GEE'.


Output()

In [ ]:
#PRUEBA
# --- ESTO VA EN TU CELDA [3] DE PRUEBAS ---

try:
    # 1. Define tus datos de prueba
    entidad_prueba = 'Ciudad de México' # O un valor que sepas que existe en tu gdf
    municipio_prueba = 'Azcapotzalco' # O un valor que sepas que existe
    ruta_copia_prueba = 'MiDrive/Copia_Prueba' # Ruta de prueba

    print(f"Probando: _obtener_roi_desde_gdf para {municipio_prueba}, {entidad_prueba}...")

    Map = geemap.Map(center=[20.5888, -100.3899], zoom=7)

    # 2. Llama a la función
    test_roi = _obtener_roi_desde_gdf(entidad_prueba, municipio_prueba, gdf)

    # 3. Verifica el resultado
    print("¡Éxito! Se generó la ROI.")
    print("Información de la geometría (GEE):")
    print(test_roi.getInfo())

    # 4. (Opcional) Visualiza el resultado
    Map.centerObject(test_roi, 10)
    Map.addLayer(test_roi, {'color': 'blue'}, 'ROI de Prueba')
    print("ROI de prueba añadida al mapa en color azul.")


    # 1. Define tus datos de prueba (necesitas una ROI y fechas)
    # (Asumimos que 'test_roi' existe del bloque anterior)

    fecha_ini_prueba = date(2023, 1, 1)
    fecha_fin_prueba = date(2023, 3, 31)

    print(f"Probando: _obtener_coleccion_landsat de {fecha_ini_prueba} a {type(fecha_fin_prueba)}...")

    # 2. Llama a la función
    test_coleccion, fecha_inicio_str, fecha_fin_str = _obtener_coleccion_landsat(test_roi, fecha_ini_prueba, fecha_fin_prueba)

    print(f"Fechas str: {fecha_inicio_str} a {type(fecha_fin_str)}...")

    # 3. Verifica el resultado
    count = test_coleccion.size().getInfo()
    print(f"¡Éxito! Se encontraron {count} imágenes.")

    # 4. Procesar TSM (Paso de Lógica de Negocio)
    resultados = _procesar_por_periodicidad(
        test_coleccion, test_roi, 'Mensual',
        fecha_ini_prueba, fecha_fin_prueba,
        municipio_prueba, ruta_copia_prueba
    )

    display(Map)

except Exception as e:
    print(f"--- ¡FALLÓ LA PRUEBA! ---")
    print(f"Error: {e}")

Probando: _obtener_roi_desde_gdf para Azcapotzalco, Ciudad de México...
--- 2. Creando ROI (ee.Geometry) ---
ROI cargada en Earth Engine.
¡Éxito! Se generó la ROI.
Información de la geometría (GEE):
{'type': 'Polygon', 'coordinates': [[[-99.20433506040008, 19.515034199670904], [-99.20467164967796, 19.515135969750972], [-99.20496051997111, 19.514964309720206], [-99.2051401800651, 19.514857549596027], [-99.20523724976387, 19.514799879767494], [-99.20552278039503, 19.5146035102649], [-99.20580961974387, 19.51440625003962], [-99.20591902877518, 19.514330999698736], [-99.20617415026503, 19.51419157971279], [-99.20659831980097, 19.51395977975766], [-99.20701611978987, 19.51373145983905], [-99.20712929025719, 19.513669620177073], [-99.20809509032239, 19.51321816967904], [-99.20814383005177, 19.51319537973455], [-99.20852372044988, 19.51295881961872], [-99.2087059906323, 19.512845323115435], [-99.20886074417913, 19.512748960781625], [-99.20897048025722, 19.51268063014897], [-99.2090732197075, 

Map(center=[19.48532844381728, -99.18210595602335], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
print(resultados)

["  -> Capa 'TSM_Mediana_Enero_2023_Azcapotzalco' añadida (visible: False).", "  -> Capa 'TSM_Mediana_Febrero_2023_Azcapotzalco' añadida (visible: False).", "  -> Capa 'TSM_Mediana_Marzo_2023_Parcial_Azcapotzalco' añadida (visible: False)."]


In [ ]:
# --- ESTO VA EN TU CELDA [3] DE PRUEBAS ---

try:
    # 1. Define tus datos de prueba (necesitas una ROI y fechas)
    # (Asumimos que 'test_roi' existe del bloque anterior)

    fecha_ini_prueba = date(2023, 1, 1)
    fecha_fin_prueba = date(2023, 3, 31)

    print(f"Probando: _obtener_coleccion_landsat de {fecha_ini_prueba} a {type(fecha_fin_prueba)}...")

    # 2. Llama a la función
    test_coleccion, fecha_inicio_str, fecha_fin_str = _obtener_coleccion_landsat(test_roi, fecha_ini_prueba, fecha_fin_prueba)

    print(f"Fechas str: {fecha_inicio_str} a {type(fecha_fin_str)}...")

    # 3. Verifica el resultado
    count = test_coleccion.size().getInfo()
    print(f"¡Éxito! Se encontraron {count} imágenes.")

except Exception as e:
    print(f"--- ¡FALLÓ LA PRUEBA! ---")
    print(f"Error: {e}")

Probando: _obtener_coleccion_landsat de 2023-01-01 a <class 'datetime.date'>...

--- 3. Filtrando Colección Landsat 8 ---
Imágenes Landsat 8 encontradas en el rango: 8
Fechas str: 2023-01-01 a <class 'str'>...
¡Éxito! Se encontraron 8 imágenes.


In [ ]:
def procesar_capa_dummy(coleccion_periodo, roi, nombre_capa, exportar_flag, target_copy_folder):
    """
    Función DUMMY para probar la lógica de bucles.
    Simplemente simula que el procesamiento fue exitoso.
    """
    count = coleccion_periodo.size().getInfo()
    print(f"   [Prueba DUMMY] Procesando '{nombre_capa}'...")
    print(f"   > Imágenes en este periodo: {count}")
    print(f"   > Exportar: {exportar_flag}, Carpeta de copia: '{target_copy_folder}'")

    # Simula el mensaje de retorno
    return f"Éxito (DUMMY): {nombre_capa} procesada con {count} imágenes."

def _procesar_por_periodicidad(coleccion, roi, periodo, start_dt, end_dt, municipio_sel, target_copy_folder):
    """
    Toma la colección completa y la procesa según la periodicidad (Semanal, Mensual, Anual).
    Llama a 'procesar_capa' para cada sub-periodo.
    """
    print(f"\n--- 4. Procesamiento de TSM por Periodo: {periodo} ---")
    resultados_procesamiento = []

    # NOTA: Asumimos que la función 'procesar_capa' existe en tu código.
    # procesar_capa(coleccion_periodo, roi, nombre_capa, exportar_flag, target_copy_folder)

    if periodo == 'Mensual':
        # --- PROCESAMIENTO MENSUAL ---
        nombres_meses = ['', 'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
        current_dt = start_dt.replace(day=1)

        while current_dt <= end_dt:
            current_year = current_dt.year
            current_month = current_dt.month
            days_in_month = calendar.monthrange(current_year, current_month)[1]
            end_filter_dt = current_dt + timedelta(days=days_in_month)

            start_date_str_p = current_dt.strftime('%Y-%m-%d')

            if end_filter_dt > end_dt:
                end_date_str_exclusive = (end_dt + timedelta(days=1)).strftime('%Y-%m-%d')
                nombre_capa = f'TSM_Mediana_{nombres_meses[current_month]}_{current_year}_Parcial_{municipio_sel}'
            else:
                end_date_str_exclusive = end_filter_dt.strftime('%Y-%m-%d')
                nombre_capa = f'TSM_Mediana_{nombres_meses[current_month]}_{current_year}_{municipio_sel}'

            coleccion_periodo = coleccion.filterDate(start_date_str_p, end_date_str_exclusive)

            if coleccion_periodo.size().getInfo() > 0:
                msg = procesar_capa_dummy(coleccion_periodo, roi, nombre_capa, False, target_copy_folder)
                resultados_procesamiento.append(msg)

            current_dt = end_filter_dt

    elif periodo == 'Anual':
        # --- PROCESAMIENTO ANUAL ---
        nombre_capa = f'TSM_Mediana_Anual_{start_dt.strftime("%Y-%m-%d")}_a_{end_dt.strftime("%Y-%m-%d")}_{municipio_sel}'
        if coleccion.size().getInfo() > 0:
            msg = procesar_capa_dummy(coleccion, roi, nombre_capa, True, target_copy_folder)
            resultados_procesamiento.append(msg)

    elif periodo == 'Semanal':
        # --- PROCESAMIENTO SEMANAL ---
        current_date = start_dt
        week_index = 1
        while current_date <= end_dt:
            start_week = current_date
            end_week_inclusive = current_date + timedelta(days=6)
            if end_week_inclusive > end_dt:
                end_week_inclusive = end_dt

            start_str_p = start_week.strftime('%Y-%m-%d')
            end_str_exclusive = (end_week_inclusive + timedelta(days=1)).strftime('%Y-%m-%d')
            end_str_inclusive = end_week_inclusive.strftime('%Y-%m-%d')

            coleccion_periodo = coleccion.filterDate(start_str_p, end_str_exclusive)

            if coleccion_periodo.size().getInfo() > 0:
                nombre_capa = f'TSM_Semana_{week_index}_{start_str_p}_a_{end_str_inclusive}_{municipio_sel}'
                msg = procesar_capa_dummy(coleccion_periodo, roi, nombre_capa, False, target_copy_folder)
                resultados_procesamiento.append(msg)

            current_date = current_date + timedelta(days=7)
            week_index += 1
    else:
        raise Exception(f"Periodicidad '{periodo}' no reconocida.")

    return resultados_procesamiento

In [ ]:
# --- ESTO VA EN TU CELDA [3] DE PRUEBAS ---
try:
    # 1. Define tus datos de prueba (los reutilizamos de pruebas anteriores)
    periodo_prueba = 'Mensual' # <-- ¡Cambia esto a 'Semanal' o 'Anual' para probarlos!
    ruta_copia_prueba = 'MiDrive/Copia_Prueba' # Ruta de prueba

    print(f"Probando: _procesar_por_periodicidad para periodo '{periodo_prueba}'...")

    # 2. Llama a la función
    # (Asegúrate de que estas variables existen de tus pruebas de celda anteriores)
    resultados = _procesar_por_periodicidad(
        test_coleccion,        # De la prueba 2
        test_roi,              # De la prueba 1
        periodo_prueba,        # El periodo que estamos probando
        fecha_ini_prueba,      # De la prueba 2
        fecha_fin_prueba,      # De la prueba 2
        municipio_prueba,      # De la prueba 1
        ruta_copia_prueba      # Ruta de prueba
    )

    # 3. Verifica el resultado
    print("\n--- ¡Éxito de la Prueba de Bucle! ---")
    print("Se ejecutó la lógica de periodicidad. Mensajes generados:")

    if not resultados:
        print("ADVERTENCIA: No se generó ningún resultado (quizás no había imágenes en los sub-periodos).")
    else:
        for res in resultados:
            print(res)

    print("\n(Recuerda: esto usó 'procesar_capa_dummy')")

except Exception as e:
    print(f"--- ¡FALLÓ LA PRUEBA! ---")
    print(f"Error: {e}")

Probando: _procesar_por_periodicidad para periodo 'Mensual'...

--- 4. Procesamiento de TSM por Periodo: Mensual ---
   [Prueba DUMMY] Procesando 'TSM_Mediana_Enero_2023_Querétaro'...
   > Imágenes en este periodo: 3
   > Exportar: False, Carpeta de copia: 'MiDrive/Copia_Prueba'
   [Prueba DUMMY] Procesando 'TSM_Mediana_Febrero_2023_Querétaro'...
   > Imágenes en este periodo: 2
   > Exportar: False, Carpeta de copia: 'MiDrive/Copia_Prueba'
   [Prueba DUMMY] Procesando 'TSM_Mediana_Marzo_2023_Parcial_Querétaro'...
   > Imágenes en este periodo: 3
   > Exportar: False, Carpeta de copia: 'MiDrive/Copia_Prueba'

--- ¡Éxito de la Prueba de Bucle! ---
Se ejecutó la lógica de periodicidad. Mensajes generados:
Éxito (DUMMY): TSM_Mediana_Enero_2023_Querétaro procesada con 3 imágenes.
Éxito (DUMMY): TSM_Mediana_Febrero_2023_Querétaro procesada con 2 imágenes.
Éxito (DUMMY): TSM_Mediana_Marzo_2023_Parcial_Querétaro procesada con 3 imágenes.

(Recuerda: esto usó 'procesar_capa_dummy')


### 1. Librerías y Google Drive

In [ ]:
# ==============================================================================
# 1. INSTALACIÓN DE LIBRERÍAS Y MONTAJE DE GOOGLE DRIVE
# ==============================================================================

# Instalar geopandas si no está disponible. El argumento -q es para instalación silenciosa.
!pip install geopandas -q

# Importar la librería para montar Google Drive.
from google.colab import drive

# Montar Google Drive en Colab.
# Al ejecutar esta línea, aparecerá un enlace o un formulario para autenticar tu cuenta.
print("Montando Google Drive. Por favor, sigue las instrucciones de autenticación.")
drive.mount('/content/drive')

# Importar las librerías necesarias para el procesamiento y la interfaz.
import geopandas as gpd
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from datetime import date # Necesario para establecer valores iniciales de fecha

Montando Google Drive. Por favor, sigue las instrucciones de autenticación.
Mounted at /content/drive


### 2. Carga de datos

In [ ]:
# ==============================================================================
# 2. CARGA DE DATOS DESDE LA UNIDAD COMPARTIDA
# ==============================================================================

# Definir la ruta del archivo GeoJSON en la Unidad Compartida (Shared Drive).
SHARED_DRIVE_NAME = "Si_IC (Isla de Calor CAMe)"
FOLDER_NAME = "Datos"
FILE_NAME = "megalopolis_mun.geojson"

FILE_PATH = f"/content/drive/Shared drives/{SHARED_DRIVE_NAME}/{FOLDER_NAME}/{FILE_NAME}"

# Cargar el archivo GeoJSON en un GeoDataFrame.
try:
    gdf = gpd.read_file(FILE_PATH)
    print("\n¡GeoDataFrame cargado exitosamente desde Google Drive!")
    print(f"Número total de municipios encontrados: {len(gdf)}")

except FileNotFoundError:
    print(f"\nERROR: El archivo no se encontró en la ruta especificada: {FILE_PATH}")
    print("Por favor, verifica el nombre de la Unidad Compartida y la ruta de la carpeta.")
    # Detener la ejecución si no se carga el archivo
    raise

# Verificar la existencia de las columnas requeridas
if 'NOM_ENT' not in gdf.columns or 'NOM_MUN' not in gdf.columns:
    raise ValueError("El GeoJSON debe contener las columnas 'NOM_ENT' y 'NOM_MUN'.")

# Mostrar las primeras filas para verificar la carga
print("\nPrimeras 5 filas del GeoDataFrame:")
display(gdf.head())


¡GeoDataFrame cargado exitosamente desde Google Drive!
Número total de municipios encontrados: 556

Primeras 5 filas del GeoDataFrame:


,CVE_ENT,CVE_MUN,CVEGEO,NOMGEO,NOM_ENT,NOM_MUN,geometry
0,09,002,09002,Azcapotzalco,Ciudad de México,Azcapotzalco,"MULTIPOLYGON (((2792538.869 838006.936, 279258..."
1,09,003,09003,Coyoacán,Ciudad de México,Coyoacán,"MULTIPOLYGON (((2796292.667 820916.951, 279638..."
2,09,004,09004,Cuajimalpa de Morelos,Ciudad de México,Cuajimalpa de Morelos,"MULTIPOLYGON (((2787186.756 825728.222, 278718..."
3,09,005,09005,Gustavo A. Madero,Ciudad de México,Gustavo A. Madero,"MULTIPOLYGON (((2800966.25 846721.791, 2801109..."
4,09,006,09006,Iztacalco,Ciudad de México,Iztacalco,"MULTIPOLYGON (((2808278.336 828072.213, 280829..."


### 3. Funciones y controles interactivos

In [ ]:
# ==============================================================================
# 3. CREACIÓN DE CONTROLES INTERACTIVOS (IPYWIDGETS)
# ==============================================================================

# Variables globales para almacenar la selección final (útiles para código posterior)
GLOBAL_SELECCION = {
    'entidad': None,
    'municipio': None,
    'fecha_inicial': None,
    'fecha_final': None,
    'periodicidad': None
}

# 3.1 Controles de Ubicación (Entidad y Municipio)
entidades_unicas = sorted(gdf['NOM_ENT'].unique())

entidad_dropdown = widgets.Dropdown(
    options=entidades_unicas,
    description='1. Entidad:',
    disabled=False,
    style={'description_width': 'initial'}
)

municipio_dropdown = widgets.Dropdown(
    options=[],
    description='2. Municipio:',
    disabled=True,
    style={'description_width': 'initial'}
)

# Función de "Observador" que mantiene la lógica de cascada (Entidad -> Municipio)
def on_entidad_change(change):
    """
    Actualiza la lista de municipios basado en la entidad seleccionada.
    """
    if change['type'] == 'change' and change['name'] == 'value':
        entidad_seleccionada = change['new']

        municipios_df = gdf[gdf['NOM_ENT'] == entidad_seleccionada]
        municipios_unicos = sorted(municipios_df['NOM_MUN'].unique())

        municipio_dropdown.options = municipios_unicos
        municipio_dropdown.disabled = False

        # Limpiar la salida cada vez que cambia la entidad
        with output:
            clear_output()

# 3.2 Controles de Temporalidad y Periodicidad
fecha_inicial_picker = widgets.DatePicker(
    value=date.today(), # Fecha inicial por defecto: hoy
    description='3. Fecha Inicial:',
    disabled=False,
    style={'description_width': 'initial'}
)

fecha_final_picker = widgets.DatePicker(
    value=date.today(), # Fecha final por defecto: hoy
    description='4. Fecha Final:',
    disabled=False,
    style={'description_width': 'initial'}
)

periodicidad_radio = widgets.RadioButtons(
    options=['Semanal', 'Mensual'],
    description='5. Periodicidad:',
    value='Semanal', # Valor por defecto
    disabled=False,
    style={'description_width': 'initial'}
)

# 3.3 Botón de Ejecución
ejecutar_button = widgets.Button(
    description="Ejecutar Visualización",
    button_style='success', # Estilo visual de éxito
    icon='check'
)

# 3.4 Contenedor de Salida
output = widgets.Output()

# Función principal que dibuja el mapa y confirma la selección (se ejecuta al presionar el botón)
def ejecutar_visualizacion(b):
    """
    Recolecta todas las selecciones de los widgets, dibuja el mapa
    e imprime la confirmación de la temporalidad.
    """

    # 1. Recolectar valores
    entidad_sel = entidad_dropdown.value
    municipio_sel = municipio_dropdown.value
    fecha_ini_sel = fecha_inicial_picker.value
    fecha_fin_sel = fecha_final_picker.value
    periodicidad_sel = periodicidad_radio.value

    # 2. Asignar a la variable global (para uso posterior)
    GLOBAL_SELECCION.update({
        'entidad': entidad_sel,
        'municipio': municipio_sel,
        'fecha_inicial': fecha_ini_sel,
        'fecha_final': fecha_fin_sel,
        'periodicidad': periodicidad_sel
    })

    # 3. Limpiar y generar la salida
    with output:
        clear_output()

        # --- LÓGICA DE PLOTEO DEL MAPA ---
        if municipio_sel and entidad_sel:
            print(f"Dibujando el mapa de: {municipio_sel}, {entidad_sel}...")

            # Filtrar el GeoDataFrame para obtener la geometría del municipio
            municipio_gdp = gdf[
                (gdf['NOM_ENT'] == entidad_sel) &
                (gdf['NOM_MUN'] == municipio_sel)
            ]

            # Crear la figura y el eje para el mapa
            fig, ax = plt.subplots(1, 1, figsize=(10, 10))

            # Dibujar el municipio seleccionado.
            municipio_gdp.plot(ax=ax, edgecolor='black', facecolor='#a1c3d1', linewidth=1.5)

            # Configurar el título y apariencia
            ax.set_title(
                f'Municipio Seleccionado: {municipio_sel} ({entidad_sel})',
                fontsize=14,
                pad=15,
                fontweight='bold'
            )
            ax.set_axis_off()
            plt.style.use('ggplot')
            plt.show()

            # --- CONFIRMACIÓN DE LA SELECCIÓN ---
            print("\n--- Confirmación de Parámetros Seleccionados ---")
            print(f"Ubicación: {municipio_sel}, {entidad_sel}")
            print(f"Fecha Inicial: {fecha_ini_sel.strftime('%Y-%m-%d') if fecha_ini_sel else 'N/A'}")
            print(f"Fecha Final: {fecha_fin_sel.strftime('%Y-%m-%d') if fecha_fin_sel else 'N/A'}")
            print(f"Periodicidad: {periodicidad_sel}")
            print("\nLa variable global GLOBAL_SELECCION ha sido actualizada con estos valores.")
        else:
            print("Por favor, selecciona una Entidad y un Municipio antes de Ejecutar.")


# Conectar las funciones "Observador" a los widgets
entidad_dropdown.observe(on_entidad_change)
ejecutar_button.on_click(ejecutar_visualizacion) # Conectar el botón a la función de visualización

## Despliegue de la interfaz

In [ ]:
# ==============================================================================
# 4. DESPLIEGUE DE LA INTERFAZ
# ==============================================================================

# Agrupar los controles en un layout vertical
controles = widgets.VBox([
    entidad_dropdown,
    municipio_dropdown,
    widgets.HBox([fecha_inicial_picker, fecha_final_picker]), # Fechas en una línea
    periodicidad_radio,
    ejecutar_button
])

print("\n--- Interfaz de Visualización y Parámetros ---")
print("Selecciona las opciones y presiona 'Ejecutar Visualización'.")
display(controles, output)

# Se inicializa el dropdown de municipio con la primera entidad para que no esté vacío al inicio
if entidades_unicas:
    on_entidad_change({'type': 'change', 'name': 'value', 'new': entidades_unicas[0]})



--- Interfaz de Visualización y Parámetros ---
Selecciona las opciones y presiona 'Ejecutar Visualización'.


Output()